### 라이브러리 설치

In [ ]:
# PyTorch (CUDA 12.1)
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121

#  Hugging Face 핵심
!pip install transformers==4.43.3 datasets==2.21.0 accelerate==0.31.0

# LoRA & RLHF (peft + trl)
!pip install peft==0.11.1 trl==0.9.6

# 전처리
!pip install scikit-learn==1.5.0 scipy==1.13.1 pandas==2.2.2 pyarrow==14.0.2 sentencepiece==0.2.0

# 로깅 & 모니터링
!pip install wandb==0.17.6 tensorboard==2.17.0

#  평가지표
!pip install rouge-score  -q
!pip install evaluate


### huggingface 모델 불러오기 위한 api key login

In [ ]:
from huggingface_hub import notebook_login


notebook_login()

## 파인튜닝 가중치 파일 만들기

lora 어뎁터 생성

In [ ]:
# 1. 라이브러리
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, get_peft_model

###### 학습시킬 데이터 (경로 수정!!) ######
csv_path = "/content/drive/MyDrive/25-2_ capstone/finetuned_edited.csv"
df = pd.read_csv(csv_path)


# 2. 여기서 원하는 모델 불러오기. 모델 다양하게 쓴다면 코드 바뀔 수 있고, 파라미터도 바뀔 수 있음
model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
print(f"'{model_id}' 모델 로드를 시작합니다...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,   # ✅ A100 → bf16
    device_map="auto",
    trust_remote_code=True
)

# 3. LoRA 설정 (수어 번역용: q,v에 집중)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # attention head 핵심 부분에 적용
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("✅ LoRA 적용 완료")



# 데이터 확인
print(f"데이터 샘플 수: {len(df)}")
print(df.head(2))

# 5. Dataset 변환
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# 6. 전처리 함수
def preprocess_function(examples):
    texts = examples["formatted"]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512,  # ✅ 문장 길이 여유 확보
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["formatted"])

# 7. Data collator (LM 목적 → MLM=False)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 8. 학습 설정
training_args = TrainingArguments(
    output_dir="./exaone-ksl",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    # num_train_epochs=3,
    # warmup_ratio=0.05,
    # lr_scheduler_type="cosine",
    # weight_decay=0.01,
    bf16=True,                     # ✅ A100이면 bf16 활성
    report_to="none",
    save_total_limit=2,
    gradient_checkpointing=False   # ✅ 메모리 절약
)

# 9. Trainer 연결
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)
model.config.use_cache = False

# 10. 학습 시작
trainer.train()

# 11. 어댑터 저장
save_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ LoRA 파인튜닝 완료! 어댑터가 저장됨: {save_path}")


'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct' 모델 로드를 시작합니다...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

✅ LoRA 적용 완료
데이터 샘플 수: 12488
                                           formatted
0  Instruction: Translate Korean to Korean Sign L...
1  Instruction: Translate Korean to Korean Sign L...


Map:   0%|          | 0/11239 [00:00<?, ? examples/s]

Map:   0%|          | 0/1249 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.904400,0.926141
2,0.712900,0.822990


✅ LoRA 파인튜닝 완료! 어댑터가 저장됨: /content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora


어뎁터 + 원본 모델 병합 파일 생성

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
adapter_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora"
final_save_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged"

# 원본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# LoRA 어댑터 병합
merged_model = PeftModel.from_pretrained(base_model, adapter_path)
merged_model = merged_model.merge_and_unload()   # 🔑 병합 + 어댑터 제거

# 최종 모델 저장 (이제 하나의 완전한 모델이 됨)
merged_model.save_pretrained(final_save_path, safe_serialization=True)
tokenizer.save_pretrained(final_save_path)

print(f"✅ 최종 병합 모델 저장 완료: {final_save_path}")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

✅ 최종 병합 모델 저장 완료: /content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged


예시로 한 문장 번역 돌려보기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
### 모델 토크나이저 같은 경로로 할당해주기!!

model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged"
)
### 디코더 출력물 확인하기.

text = "Instruction: 한국어 문장을 한국수어 글로스로 번역하시오.\nInput: 나는 경영학을 전공을 하고 있어.\nOutput:"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### 유튜브 stt 스크립트로 돌려보기(모델에 프롬프트 엔지니어링과 후처리 추가하기)

In [ ]:
import re
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer


# ✅ 수어 번역용 Instruction (여기 확장??)
def create_prompt_for_llm(text: str):
    return (
        "Instruction: Translate Korean to Korean Sign Language gloss. "
        "Follow Korean Sign Language grammar principles: "
        "omit particles and verb endings, focus on key meaning words (nouns, verbs, adjectives), "
        "use topic–comment order, and avoid redundant or filler words. "
        "Use gesture tags only when necessary.\n"
        f"Input: {text}\nOutput:"
    )


# ✅ 모델 로드
def load_llm(model_path):
    print(f"'{model_path}' 모델 로드를 시작합니다...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto"
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("✅ 모델 로드 완료")
    return model, tokenizer


# ✅ 중복 단어 제거
def remove_duplicate_words(text: str):
    text = text.replace("#", " ").replace(",", " ").strip()
    words = text.split()
    seen = set()
    unique_words = []
    for w in words:
        if w not in seen:
            seen.add(w)
            unique_words.append(w)
    return " ".join(unique_words)


# ✅ 수어 글로스 후처리 함수
def postprocess_gloss(text: str):
    text = re.sub(r"\d{8,}", "많다1", text)
    # 2️⃣ 단위 변환
    text = (
        text.replace("퍼센트", "퍼1")
            .replace("개월", "달1")
            .replace("년", "해1")
            .replace("월", "달1")
    )
    # 3️⃣ "지시1" 과다 시 등장 횟수 축소
    text = re.sub(r"(지시\d\s+){2,}", "지시1 ", text)
    # 4️⃣ "맞다" 반복 정리
    text = re.sub(r"(지시\d\s+){2,}", "지시1 ", text)
    text = re.sub(r"(맞다\d\s+){2,}", "맞다1 ", text)
    # 5️⃣ 불필요한 중복 제거
    text = remove_duplicate_words(text)
    return text.strip()


# ✅ LLM 실행 + 후처리 통합
def run_llm_on_segments(model, tokenizer, segments, max_new_tokens=64):
    results = []

    for i, seg in enumerate(segments):
        text = seg.get("text", "").strip()
        if not text:
            continue

        prompt = create_prompt_for_llm(text)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.3,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

        raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        gloss_raw = raw_output.split("Output:")[-1].strip() if "Output:" in raw_output else raw_output.strip()

        # ✅ 후처리 전체 적용
        gloss_cleaned = postprocess_gloss(gloss_raw)

        # ✅ 콘솔 출력
        print(f"\n[{i+1}/{len(segments)}] 문장:")
        print(f"🗣️ Input : {text}")
        print(f"🤖 Raw Output : {gloss_raw}")
        print(f"✨ Cleaned Output : {gloss_cleaned}")
        print("-" * 90)

        results.append({
            "file": seg.get("file"),
            "text": text,
            "start": seg.get("start"),
            "end": seg.get("end"),
            "ksl_gloss": gloss_cleaned
        })

        if (i + 1) % 10 == 0:
            torch.cuda.empty_cache()

    return results


# ✅ 메인 실행부
if __name__ == "__main__":
    transcript_path = "/content/drive/MyDrive/25-2_ capstone/mapped_transcript.json"
    model_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged"

    model, tokenizer = load_llm(model_path)

    with open(transcript_path, "r", encoding="utf-8") as f:
        segments = json.load(f)

    gloss_results = run_llm_on_segments(model, tokenizer, segments)

    with open("ksl_gloss_results_postprocessed.json", "w", encoding="utf-8") as f:
        json.dump(gloss_results, f, ensure_ascii=False, indent=2)

    print(" ksl_gloss_results_postprocessed.json 생성됨")


###  모델 선언 및 평가데이터 생성

In [ ]:
import re
import torch
import json
import pandas as pd
import numpy as np
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm


# ✅ 수어 번역 프롬프트
def create_prompt_for_llm(text: str):
    return (
        "Instruction: Translate Korean to Korean Sign Language gloss. "
        "Follow Korean Sign Language grammar principles: "
        "omit particles and verb endings, focus on key meaning words (nouns, verbs, adjectives), "
        "use topic–comment order, and avoid redundant or filler words. "
        "Use gesture tags only when necessary.\n"
        f"Input: {text}\nOutput:"
    )


# ✅ 모델 로드 (A100 최적화)
def load_llm(model_path):
    print(f"'{model_path}' 모델 로드를 시작합니다...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.model_max_length = 1024
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto"
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("✅ 모델 로드 완료 (A100 최적화)")
    return model, tokenizer


# ✅ 후처리
def remove_duplicate_words(text: str):
    text = text.replace("#", " ").replace(",", " ").strip()
    words = text.split()
    seen, unique = set(), []
    for w in words:
        if w not in seen:
            seen.add(w)
            unique.append(w)
    return " ".join(unique)

def postprocess_gloss(text: str):
    text = re.sub(r"\d{8,}", "많다1", text)
    text = (
        text.replace("퍼센트", "퍼1")
            .replace("개월", "달1")
            .replace("년", "해1")
            .replace("월", "달1")
    )
    text = re.sub(r"(지시\d\s+){2,}", "지시1 ", text)
    text = re.sub(r"(맞다\d\s+){2,}", "맞다1 ", text)
    text = remove_duplicate_words(text)
    return text.strip()


# ✅ 배치 추론
def batch_generate(model, tokenizer, texts, max_new_tokens=64):
    prompts = [create_prompt_for_llm(t) for t in texts]
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=200
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            use_cache=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]
    return [r.split("Output:")[-1].strip() if "Output:" in r else r.strip() for r in decoded]


# ✅ 메인
if __name__ == "__main__":
    model_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged"
    data_path = "/content/drive/MyDrive/25-2_ capstone/test_dataset.csv"

    model, tokenizer = load_llm(model_path)
    df = pd.read_csv(data_path)
    ### 여기서 3000개 선택했음
    df = df.head(3000).reset_index(drop=True)
    print(f"📄 총 {len(df)}개 문장 생성 시작...")

    preds = []
    start = time.time()

    for i in tqdm(range(0, len(df), 8)):  # 배치 크기 2로 OOM 방지
        batch = df.iloc[i:i+8]
        texts = batch["input"].tolist()
        preds_raw = batch_generate(model, tokenizer, texts)
        preds_post = [postprocess_gloss(p) for p in preds_raw]
        preds.extend(preds_post)
        torch.cuda.empty_cache()

    df["pred_gloss"] = preds
    print(f"✅ 총 {len(preds)}개 생성 완료. 실행 시간: {time.time()-start:.2f}s")

    output_path = "/content/drive/MyDrive/25-2_ capstone/generated_results.csv"
    df.to_csv(output_path, index=False)
    print(f"📁 결과 저장 완료: {output_path}")


### 평가진행

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import evaluate
import time


# ✅ 코사인 유사도 계산
def calc_cosine(pred, target):
    vectorizer = CountVectorizer().fit([pred, target])
    vectors = vectorizer.transform([pred, target])
    return float(cosine_similarity(vectors)[0, 1])


# ✅ 전체 평가
def evaluate_generated(file_path):
    print(f"📂 파일 로드 중: {file_path}")
    df = pd.read_csv(file_path)

    preds = df["pred_gloss"].astype(str).tolist()
    refs = df["target"].astype(str).tolist()

    start = time.time()

    # ROUGE (corpus-level)
    rouge = evaluate.load("rouge")
    results = rouge.compute(predictions=preds, references=refs, use_stemmer=True)

    # Cosine 평균
    cosines = [calc_cosine(p, r) for p, r in zip(preds, refs)]
    avg_cosine = np.mean(cosines)

    runtime = time.time() - start

    metrics = {
        "ROUGE-1": results["rouge1"] * 100,
        "ROUGE-2": results["rouge2"] * 100,
        "ROUGE-L": results["rougeL"] * 100,
        "ROUGE-Lsum": results["rougeLsum"] * 100,
        "Cosine Similarity": avg_cosine,
        "Evaluation Runtime (s)": runtime,
        "Number of Samples": len(df)
    }

    print("\n===== ✅ 평가 완료 =====")
    for k, v in metrics.items():
        print(f"{k:25s}: {v:.4f}" if isinstance(v, float) else f"{k:25s}: {v}")

    return metrics


# ✅ 실행부
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/25-2_ capstone/generated_results.csv"
    metrics = evaluate_generated(file_path)

    output_path = "/content/drive/MyDrive/25-2_ capstone/eval_results.json"
    import json
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)

    print(f"\n📁 결과 저장 완료: {output_path}")
